# **Telecom Churn Analytics:** 
## ***Building Predictive Models to Improve Customer Retention***

### Section 2 - ETL 🧹🛠️

---

### Objectives

In this section, the aim is to prepare a cleaned dataset for visualization and analysis from the raw data files. There are ETL procedures form data extraction, data cleaning and processing to data load.


### Inputs

* Datasets used for this analysis is the retail data set from Kaggle (https://www.kaggle.com/datasets/mubeenshehzadi/customer-churn-dataset/). 

* 1 raw file will be used.
    * [telecom_customer_churn.csv](../dataset/raw/telecom_customer_churn.csv) 

### Ethical Considerations

* The `customerID` field is removed to prevent exposure of personally identifiable information (PII), ensuring data privacy and compliance with regulations such as GDPR and CCPA.

* The `SeniorCitizen` and `gender` fields are initially included during exploratory analysis to understand demographic patterns influencing customer churn. These fields will be excluded from the final prediction model to prevent potential demographic bias and ensure fairness in churn prediction. This decision aligns with ethical AI principles by focusing on behavioral and service-related factors rather than personal characteristics.

### Outputs

* A cleaned dataset will be saved as a CSV file below
    * [telecom_customer_churn_cleaned.csv](../dataset/processed/telecom_customer_churn_cleaned
    .csv)
* An encoded dataset will be saved as a CSV file below
    * [telecom_customer_churn_encoded.csv](../dataset/processed/telecom_customer_churn_encoded
    .csv)




---

### Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

In [1]:
import os

os.chdir(os.path.dirname(os.getcwd()))
current_dir = os.getcwd()
print(f"You set a new current directory: {current_dir}")


You set a new current directory: /Users/denniskwok/Documents/data-analytics/telecom-churn-ml-prediction


---

## **Part A**

### **Data Extraction**

In [2]:
import numpy as np
import pandas as pd

#### Step 1: Load Dataset

In [3]:
# Load dataset from csv file
def load_csv(filepath):
    try:
        df = pd.read_csv(filepath)
        print(f"Loaded {filepath} successfully.")
        return df
    except FileNotFoundError:
        print(f"File not found: {filepath}")
        return pd.DataFrame()

df = load_csv("dataset/raw/telecom_customer_churn.csv")


Loaded dataset/raw/telecom_customer_churn.csv successfully.


### Step 2: Overview The RAW Dataset

**General dataframe information** 

In [4]:
df.info() # Display dataframe information  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


**Dataframe data overview**

In [5]:
df.head() # Display the first few rows of the dataframe

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


**Checking missing values**

In [6]:
df.isnull().sum() # Check for missing values

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

---

## **Part B**

### **Data Transformation** 

### ETL Pipeline: Telecom Churn Data Preprocessing & Feature Engineering

The preprocessing pipeline is designed to transform the raw Telecom Customer Churn dataset into a clean, machine-learning-ready format. It standardises data quality, engineers meaningful service-based features, and applies robust encoding and scaling methods to support predictive modelling.

#### **Dataset for EDA: ETL & Data Cleaning (No Scaling)**
* **Purpose:** Prepare raw data for analysis by removing noise and ensuring data integrity.

* **Main Steps:**
    - Drop personally identifiable information (`customerID`).
    - Convert `TotalCharges` to numeric and impute missing values using: `TotalCharges` = `MonthlyCharges` * `tenure`
    - Convert `SeniorCitizen` to categorical variables
    - Add derived features:
    - **CustomerType:** Phone only / Internet only / Both  
    - **NumInternetServices:** Count of active internet add-ons
    - Handle missing values with median imputation for numerical columns.

* **Output:** `telecom_customer_churn_cleaned.csv`  

#### **Dataset of ML Training: Feature Encoding & Scaling**
* **Purpose:** Prepare features for machine learning algorithms.

* **Transformations:**
    - **Numerical columns:** Standardized using `StandardScaler()`.
    - **Categorical columns:** One-hot encoded using `OneHotEncoder()` (drop first to avoid multicollinearity).

* **Output:** `telecom_customer_churn_encoded.csv`  


---


In [7]:
# =========================================================
# TELECOM CUSTOMER CHURN PIPELINE (CLEAN + ENCODED)
# =========================================================

import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

# =========================================================
# 1️⃣ Add Derived Features
# =========================================================
def add_service_features(df):
    """Add engineered features related to service usage."""
    # CustomerType: Phone only, Internet only, Both
    def service_type(row):
        if row['InternetService'] == 'No':
            return "Phone only"
        elif row['PhoneService'] == 'No':
            return "Internet only"
        else:
            return "Both"
    df['CustomerType'] = df.apply(service_type, axis=1)

    # Internet add-ons
    internet_add_ons = [
        "OnlineSecurity", "OnlineBackup", "DeviceProtection",
        "TechSupport", "StreamingTV", "StreamingMovies"
    ]

    # Normalize "No internet service" to "No"
    for col in internet_add_ons:
        df[col] = df[col].replace("No internet service", "No")

    # Count number of active internet services
    df["NumInternetServices"] = (df[internet_add_ons] == "Yes").sum(axis=1)
    return df

# =========================================================
# 2️⃣ ETL & Cleaning Stage (NO SCALING HERE)
# =========================================================
def etl_clean_data(df):
    # Add new engineered features
    df = add_service_features(df)

    # Drop customerID (PII)
    if 'customerID' in df.columns:
        df.drop("customerID", axis=1, inplace=True)

    # Convert TotalCharges to numeric
    df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")

    # Impute missing TotalCharges with domain logic
    df.loc[df['TotalCharges'].isnull(), 'TotalCharges'] = (
        df['MonthlyCharges'] * df['tenure']
    )

    # Convert SeniorCitizen to categorical
    df['SeniorCitizen'] = df['SeniorCitizen'].map({1: 'Yes', 0: 'No'})

    # Handle remaining missing values simply for EDA
    df.fillna(df.median(numeric_only=True), inplace=True)

    return df


# =========================================================
# 3️⃣ Encoding & Scaling for Modeling
# =========================================================
def build_encoded_dataset(df):
    """Apply one-hot encoding and scaling for ML model training."""
    categorical_features = df.select_dtypes(include=["object"]).columns.tolist()
    numeric_features = ["tenure", "MonthlyCharges", "TotalCharges", "NumInternetServices"]

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", StandardScaler(), numeric_features),
            ("cat", OneHotEncoder(handle_unknown="ignore", drop="first"), categorical_features)
        ]
    )

    # Transform into encoded DataFrame
    encoded_array = preprocessor.fit_transform(df)

    # Retrieve feature names
    cat_feature_names = preprocessor.named_transformers_["cat"].get_feature_names_out(categorical_features)
    all_feature_names = numeric_features + list(cat_feature_names)

    df_encoded = pd.DataFrame(encoded_array, columns=all_feature_names)

    return df_encoded, preprocessor


# =========================================================
# 4️⃣ Run the Pipeline
# =========================================================
# Load and clean
df_cleaned = etl_clean_data(df)

# Save cleaned (unscaled) dataset
df_cleaned.to_csv("dataset/processed/telecom_customer_churn_cleaned.csv", index=False)

# Encode and scale (for ML use)
df_encoded, preprocessor = build_encoded_dataset(df_cleaned)

# Save encoded dataset
df_encoded.to_csv("dataset/processed/telecom_customer_churn_encoded.csv", index=False)

print("✅ ETL + Encoding pipeline completed successfully.")
print(f"Cleaned shape: {df_cleaned.shape}")
print(f"Encoded shape: {df_encoded.shape}")
print(f"Categorical features: {len(df_cleaned.select_dtypes(include=['object']).columns)}")
print(f"Numeric features: {len(df_cleaned.select_dtypes(exclude=['object']).columns)}")
print(f"Data shape after encoding: {df_encoded.shape}")
print('\nNew Columns after One-Hot Encoding:\n')
print(df_encoded.columns.tolist(), '\n')

# Verify no missing values remain
assert df_cleaned.isnull().sum().sum() == 0, "Missing values remain in the DataFrame"




✅ ETL + Encoding pipeline completed successfully.
Cleaned shape: (7043, 22)
Encoded shape: (7043, 28)
Categorical features: 18
Numeric features: 4
Data shape after encoding: (7043, 28)

New Columns after One-Hot Encoding:

['tenure', 'MonthlyCharges', 'TotalCharges', 'NumInternetServices', 'gender_Male', 'SeniorCitizen_Yes', 'Partner_Yes', 'Dependents_Yes', 'PhoneService_Yes', 'MultipleLines_No phone service', 'MultipleLines_Yes', 'InternetService_Fiber optic', 'InternetService_No', 'OnlineSecurity_Yes', 'OnlineBackup_Yes', 'DeviceProtection_Yes', 'TechSupport_Yes', 'StreamingTV_Yes', 'StreamingMovies_Yes', 'Contract_One year', 'Contract_Two year', 'PaperlessBilling_Yes', 'PaymentMethod_Credit card (automatic)', 'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check', 'Churn_Yes', 'CustomerType_Internet only', 'CustomerType_Phone only'] 



In [8]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 28 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   tenure                                 7043 non-null   float64
 1   MonthlyCharges                         7043 non-null   float64
 2   TotalCharges                           7043 non-null   float64
 3   NumInternetServices                    7043 non-null   float64
 4   gender_Male                            7043 non-null   float64
 5   SeniorCitizen_Yes                      7043 non-null   float64
 6   Partner_Yes                            7043 non-null   float64
 7   Dependents_Yes                         7043 non-null   float64
 8   PhoneService_Yes                       7043 non-null   float64
 9   MultipleLines_No phone service         7043 non-null   float64
 10  MultipleLines_Yes                      7043 non-null   float64
 11  Inte

Remarks: 
1. After changing the data type, empty strings or invalid entries become NaN in TotolCharges column. the missing values of TotalCharges was imputed the formula assumes TotalCharges should roughly equal monthly charges multiplied by tenure.
2. Cleaned dataset without scaling and encoding for better interpretability in EDA or dashboards.

---

To be contined in Section 3 for Data Visualization.